# Creating bucket to push the model to S3

In [1]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'mlops-vit-base-human-pose-detection'
region = 'ap-south-1'

def create_bucket(bucket_name, region):
    location = {'LocationConstraint': region}
    response = s3.list_buckets()
    buckets = [bucket['Name'] for bucket in response['Buckets']]

    if bucket_name not in buckets:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f'Bucket "{bucket_name}" is created in "{region}" region.')
    else:
        print(f'Bucket "{bucket_name}" already exists.')

create_bucket(bucket_name, region)

Bucket "mlops-vit-base-human-pose-detection" is created in "ap-south-1" region.


# Push model to S3

In [2]:
import os
import boto3

s3 = boto3.client('s3')

folder_path = r'C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Human Pose Image Classification using Vision Transformers (ViT)\model'
bucket_name = 'mlops-vit-base-human-pose-detection'


In [3]:
def upload_directory_to_s3(folder_path, bucket_name, s3_prefix=""):
    # Walkeing through the directory
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Constructing full local path
            local_path = os.path.join(root, file)

            # Construct the relative path and then the full S3 key
            realtive_path = os.path.relpath(local_path, folder_path)
            s3_key = os.path.join(s3_prefix, realtive_path).replace("\\", "/")  # Replace Windows backslashes with S3-compatible slashes


            try:
                # Uploading the file to s3
                s3.upload_file(local_path, bucket_name, s3_key)
                print(f"Uploaded {local_path} to s3://{bucket_name}/{s3_key}")

            except Exception as e:
                print(f'Failed to upload {local_path}: {e}')

In [4]:
upload_directory_to_s3(folder_path, bucket_name, s3_prefix="")

Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Human Pose Image Classification using Vision Transformers (ViT)\model\config.json to s3://mlops-vit-base-human-pose-detection/config.json
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Human Pose Image Classification using Vision Transformers (ViT)\model\model.safetensors to s3://mlops-vit-base-human-pose-detection/model.safetensors
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Human Pose Image Classification using Vision Transformers (ViT)\model\preprocessor_config.json to s3://mlops-vit-base-human-pose-detection/preprocessor_config.json
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Human Pose Image Classification using Vision Transformers (ViT)\model\training_args.bin to s3://mlops-vit-base-human-pose-detection/training_args.bin


## Delete all files from S3

In [6]:
def empty_bucket(bucket_name):
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            print(f"Deleting the file '{obj['Key']}'...")
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
            print(f"The file '{obj['Key']}' has been deleted.")
    
    print('The bucket is now empty and ready to be deleted from AWS S3.')

In [7]:
empty_bucket(bucket_name)

Deleting the file 'config.json'...
The file 'config.json' has been deleted.
Deleting the file 'model.safetensors'...
The file 'model.safetensors' has been deleted.
Deleting the file 'preprocessor_config.json'...
The file 'preprocessor_config.json' has been deleted.
Deleting the file 'training_args.bin'...
The file 'training_args.bin' has been deleted.
The bucket is now empty and ready to be deleted from AWS S3.


## Delete the bucket

In [8]:
# first we need to empty files from the bucket and after that only we can delete the bucket
def delete_bucket(bucket_name):
    try:
        s3.delete_bucket(Bucket = bucket_name)
        print(f'Successfully deleted bucket "{bucket_name}"')
    except Exception as e:
        print(f'Failed to delete bucket "{bucket_name}" \nbecause of the error:"{e}"')


In [9]:
delete_bucket(bucket_name)

Successfully deleted bucket "mlops-vit-base-human-pose-detection"
